In [4]:
using CSV
using DataFrames


In [5]:
path = "../../../data/spanish-highschool"
specific_schools = ["t11_10", "t11_9", "t11_8", "t11_7", "t11_6", "t11_5", "t11_4", "t11_3", "t11_2", "t11_1", "t1", "t2", "t6"]

dfs_all2 = []
dfs_all = []
# create empty dataframe with two columns names school and class
df_index = DataFrame(school = String[], class = Int[])
for school in specific_schools
    dfs = []
    class = 0
    while true
        class += 1
        try
            fname = "Edges2_" * school * "_class_" * string(class) * ".csv"
            df = CSV.read(joinpath(path, fname), DataFrame, delim=",", header=true)
            # remove the first column
            df = select(df, names(df)[2:end])
            push!(dfs, df)
            push!(df_index, [school, class])
            # println(first(df))
        catch e
            # print(e)
            println(length(dfs))
            if length(dfs) == 0
                fname = "Edges2_" * school * "_multiple.csv"
                df = CSV.read(joinpath(path, fname), DataFrame, delim=",", header=true)
                df = select(df, names(df)[2:end])
                push!(dfs, df)

                push!(df_index, [school, class])
            end
            break
        end
    end
    push!(dfs_all, dfs)
    append!(dfs_all2, dfs)
end

# first(df)

4
4
4
1
1
4
4
2
4
2
0
0
0


In [6]:
length(dfs_all2)

33

In [7]:
# get all edges
edges_all = [[(df.source[i], df.target[i]) for i in 1:size(df, 1)] for df in dfs_all2]

# edges[1:5]

# print([length.(unique.(edges_all)), length.(edges_all)])


# remove self loops
edges_all = [[(u, v) for (u, v) in edges if u != v] for edges in edges_all]

df_index.edge_count = length.(edges_all)
df_index.unique_edge_count = length.(unique.(edges_all))

df_index

Row,school,class,edge_count,unique_edge_count
,String,Int64,Int64,Int64
1,t11_10,1,1605,1487
2,t11_10,2,1919,1760
3,t11_10,3,1670,1545
4,t11_10,4,1435,1325
5,t11_9,1,915,838
6,t11_9,2,1403,1312
7,t11_9,3,864,783
8,t11_9,4,1013,943
9,t11_8,1,616,568


In [8]:
# check if for repeated edges the sign is the same

# get all edges
edges_sign_all = [[(df.source[i], df.target[i], df.sign[i]) for i in 1:size(df, 1)] for df in dfs_all2]

# edges_sign = unique(edges_sign)

# remove self loops
edges_sign_all = [[(u, v, s) for (u, v, s) in edges_sign if u != v] for edges_sign in edges_sign_all]


33-element Vector{Vector{Tuple{Int64, Int64, Int64}}}:
 [(5143, 5232, 1), (5143, 5242, 1), (5143, 5248, 1), (5143, 5257, 1), (5143, 5234, 1), (5143, 5241, 1), (5143, 5237, 1), (5143, 5234, 1), (5143, 5235, 1), (5143, 5241, 1)  …  (5258, 5229, 1), (5258, 5234, 1), (5258, 5236, 1), (5258, 5242, 1), (5258, 5243, 1), (5258, 5231, -1), (5258, 5233, -1), (5258, 5249, -1), (5258, 5241, -1), (5258, 5231, -1)]
 [(5259, 5355, 1), (5259, 5349, 1), (5259, 5346, 1), (5259, 5349, 1), (5259, 5347, -1), (5259, 5353, -1), (5259, 5347, -1), (5260, 5266, 1), (5260, 5274, 1), (5260, 5284, 1)  …  (5370, 5348, 1), (5370, 5348, 1), (5370, 5354, 1), (5370, 5357, 1), (5370, 5363, 1), (5370, 5367, 1), (5370, 5347, -1), (5370, 5364, -1), (5370, 5356, -1), (5370, 5360, -1)]
 [(5372, 5440, 1), (5372, 5469, 1), (5372, 5450, 1), (5372, 5468, 1), (5372, 5438, 1), (5372, 5409, 1), (5372, 5399, 1), (5372, 5414, 1), (5372, 5428, 1), (5372, 5451, 1)  …  (5487, 5407, 1), (5487, 5386, 1), (5487, 5399, 1), (5487, 5443, -1),

In [9]:
# get list of all nodes
nodes_all = [unique([[u for (u, v) in edges_sign]..., [v for (u, v) in edges_sign]...]) for edges_sign in edges_sign_all]

df_index.node_count = length.(nodes_all)
# print(length(nodes))

# filter out self loops
dfs_all2_filtered = [filter(x -> x.source != x.target, df) for df in dfs_all2]

# get list of outgoing edges for each node by grouping by source
node_outs_gbf_all = [groupby(df_filtered, :source) for df_filtered in dfs_all2_filtered]

# get list of outgoing edges for each node by collecting targets
node_outs_all = [combine(node_outs_gbf, x -> [x[!, :target]]) for node_outs_gbf in node_outs_gbf_all]
# rename!(node_outs, :x1 => :targets)
for node_outs in node_outs_all
    rename!(node_outs, :x1 => :targets)
end
# map!((x) -> rename!(x, :x1 => :targets), node_outs_all)

# create a dictionary with the outgoing edges for each node
node_outs_dict_all = [Dict(zip(node_outs.source, unique.(node_outs.targets))) for node_outs in node_outs_all]

node_outs_dict_all[1]

Dict{Int64, Vector{Int64}} with 94 entries:
  5239 => [5258, 5238, 5246, 5235, 5234, 5245, 5231, 5253, 5243, 5232, 5257, 52…
  5195 => [5249, 5217, 5179, 5235, 5248, 5159, 5257, 5187, 5146, 5162  …  5236,…
  5235 => [5152, 5175, 5246, 5213, 5170, 5154, 5149, 5240, 5222, 5189  …  5184,…
  5152 => [5231, 5232, 5229, 5235, 5257, 5234, 5243, 5258]
  5161 => [5154, 5198, 5155, 5158, 5189, 5177, 5235, 5185, 5190, 5151, 5150, 51…
  5233 => [5231, 5232, 5237, 5240, 5249, 5243, 5217, 5242, 5246, 5225, 5245]
  5175 => [5241, 5246, 5235, 5248, 5257, 5234, 5245, 5143, 5210, 5194  …  5229,…
  5158 => [5246]
  5232 => [5258, 5233, 5238, 5246, 5235, 5229, 5254, 5234, 5245, 5241, 5231, 52…
  5143 => [5232, 5242, 5248, 5257, 5234, 5241, 5237, 5235, 5243, 5236, 5238, 52…
  5184 => [5193, 5152, 5188, 5148, 5191, 5204, 5165, 5169, 5170, 5171, 5175, 51…
  5243 => [5238, 5246, 5235, 5213, 5170, 5245, 5241, 5257, 5176, 5242  …  5206,…
  5223 => [5217, 5222, 5224, 5202, 5221, 5249, 5213, 5150, 5149, 5151, 516

In [10]:
df_index

Row,school,class,edge_count,unique_edge_count,node_count
,String,Int64,Int64,Int64,Int64
1,t11_10,1,1605,1487,94
2,t11_10,2,1919,1760,98
3,t11_10,3,1670,1545,99
4,t11_10,4,1435,1325,95
5,t11_9,1,915,838,67
6,t11_9,2,1403,1312,94
7,t11_9,3,864,783,75
8,t11_9,4,1013,943,83
9,t11_8,1,616,568,52


(5239, 5258, 5253)

In [14]:
# get transitive triads

triads_s = []
for node_outs_dict in node_outs_dict_all
    triads = []
    for (source, targets) in node_outs_dict
        for target in targets
            if target in keys(node_outs_dict)
                for target_target in node_outs_dict[target]
                    if target_target in targets
                        append!(triads, [(source, target, target_target)])
                    end
                end
            end
        end
    end
    push!(triads_s, triads)
end
# for (source, targets) in node_outs_dict
#     for target in targets
#         if target in keys(node_outs_dict)
#             for target_target in node_outs_dict[target]
#                 if target_target in targets
#                     append!(triads, [(source, target, target_target)])
#                 end
#             end
#         end
#     end
# end
length.(triads_s), length.(unique.(triads_s))

([9598, 14186, 9564, 7494, 5246, 7569, 3175, 4240, 3084, 2213  …  19904, 7735, 34789, 12537, 10103, 7551, 26585, 45760, 24365, 143112], [9598, 14186, 9564, 7494, 5246, 7569, 3175, 4240, 3084, 2213  …  19904, 7735, 34789, 12537, 10103, 7551, 26585, 45760, 24365, 143112])

In [18]:
length.(unique.(triads_s))[15:end]

19-element Vector{Int64}:
   7527
  11108
   9451
   8737
   3212
   3798
   3172
   1376
  12077
  19904
   7735
  34789
  12537
  10103
   7551
  26585
  45760
  24365
 143112

In [20]:
# get cyclic triads
# thanks to sorting, each such triad will be counted only once

source_nodes_s = [sort(collect(keys(node_outs_dict))) for node_outs_dict in node_outs_dict_all]

cyclic_triads_s = []
for (source_nodes, node_outs_dict) in zip(source_nodes_s, node_outs_dict_all)
    cyclic_triads = []
    for source in source_nodes
        targets = sort(node_outs_dict[source])
        # filter out targets that have smaller id than source
        targets = [target for target in targets if target > source]
        for target in targets
            if target in keys(node_outs_dict)
                target_targets = sort(node_outs_dict[target])
                # filter out target_targets that have smaller id than source
                target_targets = [target_target for target_target in target_targets if target_target > source]
                # I hope above is correct
                # target_targets = node_outs_dict[target]
                for target_target in target_targets
                    if target_target in keys(node_outs_dict)
                        if source in node_outs_dict[target_target]
                            append!(cyclic_triads, [(source, target, target_target)])
                        end
                    end
                end
            end
        end
    end
    push!(cyclic_triads_s, cyclic_triads)
end
# for source in source_nodes
#     targets = sort(node_outs_dict[source])
#     # filter out targets that have smaller id than source
#     targets = [target for target in targets if target > source]
#     for target in targets
#         if target in keys(node_outs_dict)
#             target_targets = sort(node_outs_dict[target])
#             # filter out target_targets that have smaller id than source
#             target_targets = [target_target for target_target in target_targets if target_target > source]
#             # I hope above is correct
#             # target_targets = node_outs_dict[target]
#             for target_target in target_targets
#                 if target_target in keys(node_outs_dict)
#                     if source in node_outs_dict[target_target]
#                         append!(cyclic_triads, [(source, target, target_target)])
#                     end
#                 end
#             end
#         end
#     end
# end
length.(cyclic_triads_s)

33-element Vector{Int64}:
  2069
  2524
  1577
  1550
  1216
  1639
   610
   756
   625
   527
     ⋮
  1663
  6933
  2720
  2132
  1771
  5690
 10398
  5569
 32411

In [20]:
cyclic_triads[end]  

(7666, 7691, 6946)

In [23]:
df_triads_s = [DataFrame(triads) for triads in triads_s]
for df_triads in df_triads_s
    rename!(df_triads, [:P, :O, :X]);
end
# rename!(df_triads, [:P, :O, :X]);

po_edges_s = [[(df_triads.P[i], df_triads.O[i]) for i in 1:size(df_triads, 1)] for df_triads in df_triads_s]
ox_edges_s = [[(df_triads.O[i], df_triads.X[i]) for i in 1:size(df_triads, 1)] for df_triads in df_triads_s]
px_edges_s = [[(df_triads.P[i], df_triads.X[i]) for i in 1:size(df_triads, 1)] for df_triads in df_triads_s];

In [24]:
df_cyclic_triads_s = [DataFrame(cyclic_triads) for cyclic_triads in cyclic_triads_s]
for df_cyclic_triads in df_cyclic_triads_s
    rename!(df_cyclic_triads, [:P, :O, :X]);
end
# rename!(df_cyclic_triads, [:P, :O, :X]);

po_edges_cyclic_s = [[(df_cyclic_triads.P[i], df_cyclic_triads.O[i]) for i in 1:size(df_cyclic_triads, 1)] for df_cyclic_triads in df_cyclic_triads_s]
ox_edges_cyclic_s = [[(df_cyclic_triads.O[i], df_cyclic_triads.X[i]) for i in 1:size(df_cyclic_triads, 1)] for df_cyclic_triads in df_cyclic_triads_s]
xp_edges_cyclic_s = [[(df_cyclic_triads.X[i], df_cyclic_triads.P[i]) for i in 1:size(df_cyclic_triads, 1)] for df_cyclic_triads in df_cyclic_triads_s];


In [25]:
df_all_triads_s = [DataFrame([triads..., cyclic_triads...]) for (triads, cyclic_triads) in zip(triads_s, cyclic_triads_s)]
for df_all_triads in df_all_triads_s
    rename!(df_all_triads, [:P, :O, :X]);
end
# rename!(df_all_triads, [:P, :O, :X]);
nrow.(df_all_triads_s)
# po_edges_all = [(df_all_triads.P[i], df_all_triads.O[i]) for i in 1:size(df_all_triads, 1)]

33-element Vector{Int64}:
  11667
  16710
  11141
   9044
   6462
   9208
   3785
   4996
   3709
   2740
      ⋮
   9398
  41722
  15257
  12235
   9322
  32275
  56158
  29934
 175523

In [28]:
# get sign of edges in triads. Take the last value of edge

for (i, (edges_sign, df_triads)) in enumerate(zip(edges_sign_all, df_triads_s))
    df_triads[!, "po_sign"] = [edges_sign[findall([edge] .== edges_all[i])][end][3] for edge in po_edges_s[i]]
    df_triads[!, "ox_sign"] = [edges_sign[findall([edge] .== edges_all[i])][end][3] for edge in ox_edges_s[i]]
    df_triads[!, "px_sign"] = [edges_sign[findall([edge] .== edges_all[i])][end][3] for edge in px_edges_s[i]]
end

# df_triads[!, "po_sign"] = [edges_sign[findall([edge] .== edges)][end][3] for edge in po_edges]
# df_triads[!, "ox_sign"] = [edges_sign[findall([edge] .== edges)][end][3] for edge in ox_edges]
# df_triads[!, "px_sign"] = [edges_sign[findall([edge] .== edges)][end][3] for edge in px_edges]
first(df_triads_s[1], 5)

Row,P,O,X,po_sign,ox_sign,px_sign
,Int64,Int64,Int64,Int64,Int64,Int64
1,5239,5258,5253,1,1,1
2,5239,5258,5229,1,1,1
3,5239,5258,5234,1,1,1
4,5239,5258,5236,1,1,1
5,5239,5258,5242,1,1,1


In [30]:
# get sign of edges in cyclic triads. Take the last value of edge

for (i, (edges_sign, df_cyclic_triads)) in enumerate(zip(edges_sign_all, df_cyclic_triads_s))
    df_cyclic_triads[!, "po_sign"] = [edges_sign[findall([edge] .== edges_all[i])][end][3] for edge in po_edges_cyclic_s[i]]
    df_cyclic_triads[!, "ox_sign"] = [edges_sign[findall([edge] .== edges_all[i])][end][3] for edge in ox_edges_cyclic_s[i]]
    df_cyclic_triads[!, "xp_sign"] = [edges_sign[findall([edge] .== edges_all[i])][end][3] for edge in xp_edges_cyclic_s[i]]
end

first(df_cyclic_triads_s[1], 5)

Row,P,O,X,po_sign,ox_sign,xp_sign
,Int64,Int64,Int64,Int64,Int64,Int64
1,5143,5232,5234,-1,1,1
2,5143,5232,5235,-1,1,1
3,5143,5234,5165,1,1,1
4,5143,5234,5170,1,1,1
5,5143,5234,5171,1,1,1


In [31]:
# get sign of edges in all triads. Take the last value of edge

for (df_all_triads, df_triads, df_cyclic_triads) in zip(df_all_triads_s, df_triads_s, df_cyclic_triads_s)
    df_all_triads[!, "po_sign"] = [df_triads[!, "po_sign"]..., df_cyclic_triads[!, "po_sign"]...]
    df_all_triads[!, "ox_sign"] = [df_triads[!, "ox_sign"]..., df_cyclic_triads[!, "ox_sign"]...]
    df_all_triads[!, "xpx_sign"] = [df_triads[!, "px_sign"]..., df_cyclic_triads[!, "xp_sign"]...]
end

# df_all_triads[!, "po_sign"] = [df_triads[!, "po_sign"]..., df_cyclic_triads[!, "po_sign"]...]
# df_all_triads[!, "ox_sign"] = [df_triads[!, "ox_sign"]..., df_cyclic_triads[!, "ox_sign"]...]
# df_all_triads[!, "xpx_sign"] = [df_triads[!, "px_sign"]..., df_cyclic_triads[!, "xp_sign"]...]

first(df_all_triads_s[1], 5)

Row,P,O,X,po_sign,ox_sign,xpx_sign
,Int64,Int64,Int64,Int64,Int64,Int64
1,5239,5258,5253,1,1,1
2,5239,5258,5229,1,1,1
3,5239,5258,5234,1,1,1
4,5239,5258,5236,1,1,1
5,5239,5258,5242,1,1,1


In [32]:
# save df_triads
# CSV.write(joinpath(path, "soc-wiki-elec-triads.csv"), df_triads)

In [33]:
# save df_cyclic_triads
# CSV.write(joinpath(path, "soc-wiki-elec-cyclic-triads.csv"), df_cyclic_triads)

In [34]:
# save df_all_triads
# CSV.write(joinpath(path, "soc-wiki-elec-all-triads.csv"), df_all_triads)

In [35]:
# df_triads = CSV.read(joinpath(path, "soc-wiki-elec-triads.csv"), DataFrame)

# triads = [(df_triads.P[i], df_triads.O[i], df_triads.X[i]) for i in 1:size(df_triads, 1)]

In [36]:
# df_cyclic_triads = CSV.read(joinpath(path, "soc-wiki-elec-cyclic-triads.csv"), DataFrame)

# cyclic_triads = [(df_cyclic_triads.P[i], df_cyclic_triads.O[i], df_cyclic_triads.X[i]) for i in 1:size(df_cyclic_triads, 1)]

In [37]:
# df_all_triads = CSV.read(joinpath(path, "soc-wiki-elec-all-triads.csv"), DataFrame)

In [38]:
# identify triads that consist of the same edges

all_triads_s = []
all_triads_ordered_s = []
all_triads_unique_s = []
all_triads_dict_s = []
for (triads, cyclic_triads) in zip(triads_s, cyclic_triads_s)
    all_triads = [triads..., cyclic_triads...]
    push!(all_triads_s, all_triads)

    all_triads_ordered = sort.([[p, o, x] for (p, o, x) in all_triads])
    push!(all_triads_ordered_s, all_triads_ordered)

    all_triads_unique = unique(all_triads_ordered)
    push!(all_triads_unique_s, all_triads_unique)

    # create a dictionary with unique triads as keys and indices of their appearance as values
    all_triads_dict = Dict(all_triads_unique .=> [findall(all_triads_ordered .== [triad]) for triad in all_triads_unique])
    push!(all_triads_dict_s, all_triads_dict)
end

# all_triads = [triads..., cyclic_triads...]

# all_triads_ordered = sort.([[p, o, x] for (p, o, x) in all_triads])
# all_triads_unique = unique(all_triads_ordered)

# # create a dictionary with unique triads as keys and indices of their appearance as values
# all_triads_dict = Dict(all_triads_unique .=> [findall(all_triads_ordered .== [triad]) for triad in all_triads_unique])



In [39]:
using JLD2, FileIO

save(joinpath(path, "spanish-all-triads-dict.jld2"), "all_triads_dict_s", all_triads_dict_s)

In [40]:
# create dataframe from dict



df_all_triads_dict_s = [DataFrame(triad = collect(keys(all_triads_dict)), inds = collect(values(all_triads_dict)),
                           count = length.(collect(values(all_triads_dict)))) for all_triads_dict in all_triads_dict_s]

# collect(keys(triads_dict))
df_all_triads_dict_s[1]

Row,triad,inds,count
,Array…,Array…,Int64
1,"[5143, 5236, 5238]","[2185, 2186]",2
2,"[5175, 5200, 5217]","[1607, 1780, 2801, 10235]",4
3,"[5154, 5192, 5198]",[9899],1
4,"[5206, 5209, 5222]",[7007],1
5,"[5177, 5243, 5249]",[2408],1
6,"[5168, 5190, 5257]",[4400],1
7,"[5149, 5163, 5170]",[4565],1
8,"[5175, 5194, 5223]","[1558, 1575]",2
9,"[5195, 5229, 5258]","[332, 445]",2


In [42]:
# add columns with the type of triad (transitive or cyclic) and inds in the source dataframe

# length(triads), length(cyclic_triads), length(all_triads)

for (i, df_all_triads_dict) in enumerate(df_all_triads_dict_s)
    df_all_triads_dict[!, "triad_type"] = [(inds .> length(triads_s[i])) .+ 1 for inds in df_all_triads_dict.inds]

    df_all_triads_dict[!, "exact_inds"] = [inds .- length(triads_s[i]) .* (type .- 1) for (inds, type) in zip(df_all_triads_dict.inds, df_all_triads_dict.triad_type)]
end

# df_all_triads_dict[!, "triad_type"] = [(inds .> length(triads)) .+ 1 for inds in df_all_triads_dict.inds]

# df_all_triads_dict[!, "exact_inds"] = [inds .- length(triads) .* (type .- 1) for (inds, type) in zip(df_all_triads_dict.inds, df_all_triads_dict.triad_type)]

first(df_all_triads_dict_s[1], 5)

Row,triad,inds,count,triad_type,exact_inds
,Array…,Array…,Int64,Array…,Array…
1,"[5143, 5236, 5238]","[2185, 2186]",2,"[1, 1]","[2185, 2186]"
2,"[5175, 5200, 5217]","[1607, 1780, 2801, 10235]",4,"[1, 1, 1, 2]","[1607, 1780, 2801, 637]"
3,"[5154, 5192, 5198]",[9899],1,[2],[301]
4,"[5206, 5209, 5222]",[7007],1,[1],[7007]
5,"[5177, 5243, 5249]",[2408],1,[1],[2408]


### Balance and hierarchy analysis including cyclic triads

In [43]:
get_transitive_edges(triad) = [(triad[1], triad[2]), (triad[2], triad[3]), (triad[1], triad[3])]
get_cyclic_edges(triad) = [(triad[1], triad[2]), (triad[2], triad[3]), (triad[3], triad[1])]
function get_edges(triad, type)
    if type == 1
        return get_transitive_edges(triad)
    elseif type == 2
        return get_cyclic_edges(triad)
    end
end

get_edges (generic function with 1 method)

In [46]:
using PyCall

net = 1
df_all_triads_dicti = df_all_triads_dict_s[net]
triadsi = triads_s[net]
cyclic_triadsi = cyclic_triads_s[net]
df_all_triadsi = df_all_triads_s[net]

i = 2
triad = df_all_triads_dicti.triad[i]
types = df_all_triads_dicti.triad_type[i]
inds = df_all_triads_dicti.inds[i]
exact_inds = df_all_triads_dicti.exact_inds[i]

# get triad list
sources = [triadsi, cyclic_triadsi]
type_sources = sources[types]
# triad_list = sources[types][exact_inds]
triad_list = [triad_source[exact_ind] for (triad_source, exact_ind) in zip(type_sources, exact_inds)]
# triad_list = triads[inds]

edges_list = get_edges.(triad_list, types)
edges_list = reduce(vcat, edges_list)
unique_edges = unique(edges_list)

py"""
import networkx as nx
"""

G = py"nx.DiGraph"([edges_list...])

# print(G.edges)

triad_census = py"nx.triadic_census"(G)

triad_census_triads = collect(keys(triad_census))

tc_ind = collect(values(triad_census)) .== 1

println(triad_census_triads[tc_ind][1])

# get signed triad type
sign_vals_s = values(df_all_triadsi[inds, 4:6])
sign_vals_s = Matrix(sign_vals_s)
sign_vals_s = mapslices(x->[x], sign_vals_s, dims=2)[:]

n_arr = zeros(Int, 8)

for sign_vals in sign_vals_s
    connections_values = sign_vals .== 1
    n_arr[connections_values[1]*4 + connections_values[3]*2 + connections_values[2]+1] += 1
end
n_arr
# n_arr

210


8-element Vector{Int64}:
 1
 1
 1
 0
 0
 0
 1
 0

In [97]:
triad_census_triads[tc_ind][1]

"120C"

In [48]:
using PyCall
py"""
import networkx as nx
"""

G = py"nx.DiGraph"()

triad_census = py"nx.triadic_census"(G)
triad_census_triads = collect(keys(triad_census))

# signed triad types:
n_arr_dict_s = [Dict( triad_census_triads .=> [zeros(Int, 12) for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
# first 8 as standard n_arr_dict, the next 4 from cycles: 9 - 0 neg links, 10 - 1 neg link etc.
n_bal_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
n_hier_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
n_tot_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]

n_trans_bal_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
n_trans_hier_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
n_trans_tot_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]

n_cyclic_bal_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
n_cyclic_hier_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]
n_cyclic_tot_dict_s = [Dict( triad_census_triads .=> [0 for _ in 1:length(triad_census_triads)]) for _ in 1:length(triads_s)]

inform_after = 10000
counter = 0
for (i, df_all_triads_dict) in enumerate(df_all_triads_dict_s)
    sources = [triads_s[i], cyclic_triads_s[i]]
    
    for (triad, inds, count, trans_or_cyclics, exact_inds) in eachrow(df_all_triads_dict)
        type_sources = sources[trans_or_cyclics]
        triad_list = [triad_source[exact_ind] for (triad_source, exact_ind) in zip(type_sources, exact_inds)]

        # get edges list from triad list
        edges_list = get_edges.(triad_list, trans_or_cyclics)
        edges_list = reduce(vcat, edges_list)
        unique_edges = unique(edges_list)

        # call Python from julia, use networkx to get triad census on subgraphs
        G = py"nx.DiGraph"([unique_edges...])
        triad_census = py"nx.triadic_census"(G)
        triad_census_triads = collect(keys(triad_census))
        tc_ind = collect(values(triad_census)) .== 1
        triad_type = triad_census_triads[tc_ind][1]

        # get signed triad type
        sign_vals_s = values(df_all_triads_s[i][inds, 4:6])
        sign_vals_s = Matrix(sign_vals_s)
        sign_vals_s = mapslices(x->[x], sign_vals_s, dims=2)[:]

        hier = true
        hier_trans = true
        hier_cyclic = true
        bal_trans = true
        bal_cyclic = true
        # indicators that trans/cyclic triads were considered at all
        some_trans = false
        some_cyclic = false
        for (sign_vals, trans_or_cyclic) in zip(sign_vals_s, trans_or_cyclics)
            if trans_or_cyclic == 1
                some_trans = true
                n_trans_tot_dict_s[i][triad_type] += 1
                if prod(sign_vals) != 1
                    bal_trans = false
                end

                connections_values = sign_vals .== 1
                signed_triad_type = connections_values[1]*4 + connections_values[3]*2 + connections_values[2]+1
                
            elseif trans_or_cyclic == 2
                some_cyclic = true

                n_cyclic_tot_dict_s[i][triad_type] += 1
                if prod(sign_vals) != 1
                    bal_cyclic = false
                end

                num_pos_links = sum(sign_vals .== 1)
                signed_triad_type = 9 + num_pos_links
            else
                println(trans_or_cyclics)
                error("Wrong type")
            end
            n_arr_dict_s[i][triad_type][signed_triad_type] += 1

            if signed_triad_type in [3, 6, 9, 12]
                hier = false
                if signed_triad_type in [3, 6]
                    hier_trans = false
                else
                    hier_cyclic = false
                end
            end
        end
        n_tot_dict_s[i][triad_type] += 1
        n_bal_dict_s[i][triad_type] += all(prod.(sign_vals_s) .== 1)
        n_hier_dict_s[i][triad_type] += hier

        n_trans_bal_dict_s[i][triad_type] += bal_trans * some_trans
        n_cyclic_bal_dict_s[i][triad_type] += bal_cyclic * some_cyclic

        n_trans_hier_dict_s[i][triad_type] += hier_trans * some_trans
        n_cyclic_hier_dict_s[i][triad_type] += hier_cyclic * some_cyclic

        # println(triad_list, inds)
        counter += 1
        # if counter > 5000
        #     break
        # end
        if counter % inform_after == 0
            # print time
            # print()
            # break
            println("Processed $counter triads. Currently in $i-th network")
            flush(stdout)
        # elseif counter > 23000
        #     println("Processed $counter triads")
        # else
            # break
        end
    end
end

Processed 10000 triads. Currently in 2-th network
Processed 20000 triads. Currently in 4-th network
Processed 30000 triads. Currently in 8-th network
Processed 40000 triads. Currently in 14-th network
Processed 50000 triads. Currently in 17-th network
Processed 60000 triads. Currently in 20-th network
Processed 70000 triads. Currently in 24-th network
Processed 80000 triads. Currently in 25-th network
Processed 90000 triads. Currently in 26-th network
Processed 100000 triads. Currently in 26-th network
Processed 110000 triads. Currently in 28-th network
Processed 120000 triads. Currently in 30-th network
Processed 130000 triads. Currently in 31-th network
Processed 140000 triads. Currently in 31-th network
Processed 150000 triads. Currently in 32-th network
Processed 160000 triads. Currently in 33-th network
Processed 170000 triads. Currently in 33-th network
Processed 180000 triads. Currently in 33-th network
Processed 190000 triads. Currently in 33-th network
Processed 200000 triads.

In [51]:
# create dataframe from dicts

df_cyclic_n_arr_s = [DataFrame(triad = collect(keys(n_arr_dict)), n_arr = collect(values(n_arr_dict)),
                     n_bal = collect(values(n_bal_dict)), n_hier = collect(values(n_hier_dict)),
                     n_tot = collect(values(n_tot_dict)), n_trans_bal = collect(values(n_trans_bal_dict)), 
                     n_trans_hier = collect(values(n_trans_hier_dict)), n_trans_tot = collect(values(n_trans_tot_dict)), 
                     n_cyclic_bal = collect(values(n_cyclic_bal_dict)), n_cyclic_hier = collect(values(n_cyclic_hier_dict)),
                        n_cyclic_tot = collect(values(n_cyclic_tot_dict))) for (n_arr_dict, n_bal_dict, n_hier_dict, n_tot_dict, 
                        n_trans_bal_dict, n_trans_hier_dict, n_trans_tot_dict, n_cyclic_bal_dict, n_cyclic_hier_dict, n_cyclic_tot_dict) in
                        zip(n_arr_dict_s, n_bal_dict_s, n_hier_dict_s, n_tot_dict_s, n_trans_bal_dict_s, n_trans_hier_dict_s, 
                        n_trans_tot_dict_s, n_cyclic_bal_dict_s, n_cyclic_hier_dict_s, n_cyclic_tot_dict_s)]

df_cyclic_n_arr_s[1]

Row,triad,n_arr,n_bal,n_hier,n_tot,n_trans_bal,n_trans_hier,n_trans_tot,n_cyclic_bal,n_cyclic_hier,n_cyclic_tot
,String,Array…,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,030C,"[0, 0, 0, 0, 0, 0, 0, 0, 4, 21, 31, 13]",34,52,69,0,0,0,34,52,69
2,111U,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
3,120D,"[37, 307, 15, 91, 23, 83, 71, 503, 0, 0, 0, 0]",394,470,565,394,470,1130,0,0,0
4,120C,"[7, 29, 22, 48, 21, 11, 128, 192, 2, 69, 216, 171]",217,256,458,264,425,458,240,285,458
5,201,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
6,111D,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
7,210,"[28, 111, 52, 193, 69, 193, 302, 1932, 4, 72, 278, 606]",598,165,960,598,732,2880,678,350,960
8,003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
9,012,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0


In [52]:
for df_cyclic_n_arr in df_cyclic_n_arr_s
        
    df_cyclic_n_arr.per_bal = df_cyclic_n_arr.n_bal ./ df_cyclic_n_arr.n_tot
    df_cyclic_n_arr.per_hier = df_cyclic_n_arr.n_hier ./ df_cyclic_n_arr.n_tot
    df_cyclic_n_arr.per_trans_bal = df_cyclic_n_arr.n_trans_bal ./ df_cyclic_n_arr.n_tot
    df_cyclic_n_arr.per_trans_hier = df_cyclic_n_arr.n_trans_hier ./ df_cyclic_n_arr.n_tot
    df_cyclic_n_arr.per_cyclic_bal = df_cyclic_n_arr.n_cyclic_bal ./ df_cyclic_n_arr.n_tot
    df_cyclic_n_arr.per_cyclic_hier = df_cyclic_n_arr.n_cyclic_hier ./ df_cyclic_n_arr.n_tot
    df_cyclic_n_arr.per_trans_bal_sep = [sum(n_arr[[2, 3, 5, 8]]) / n_tot for (n_arr, n_tot) in zip(df_cyclic_n_arr.n_arr, df_cyclic_n_arr.n_trans_tot)]
    df_cyclic_n_arr.per_cyclic_bal_sep = [sum(n_arr[[10, 12]]) / n_tot for (n_arr, n_tot) in zip(df_cyclic_n_arr.n_arr, df_cyclic_n_arr.n_cyclic_tot)]
    df_cyclic_n_arr.per_trans_hier_sep = [1 - sum(n_arr[[3,6]]) / n_tot for (n_arr, n_tot) in zip(df_cyclic_n_arr.n_arr, df_cyclic_n_arr.n_trans_tot)]
    df_cyclic_n_arr.per_cyclic_hier_sep = [sum(n_arr[[10, 11]]) / n_tot for (n_arr, n_tot) in zip(df_cyclic_n_arr.n_arr, df_cyclic_n_arr.n_cyclic_tot)]
    df_cyclic_n_arr.per_bal_sep = [sum(n_arr[[2, 3, 5, 8, 10, 12]]) / n_tot for (n_arr, n_tot) in zip(df_cyclic_n_arr.n_arr, df_cyclic_n_arr.n_trans_tot .+ df_cyclic_n_arr.n_cyclic_tot)]
    df_cyclic_n_arr.per_hier_sep = [1 - sum(n_arr[[3, 6, 9, 12]]) / n_tot for (n_arr, n_tot) in zip(df_cyclic_n_arr.n_arr, df_cyclic_n_arr.n_trans_tot .+ df_cyclic_n_arr.n_cyclic_tot)]
end
df_cyclic_n_arr_s[1]

Row,triad,n_arr,n_bal,n_hier,n_tot,n_trans_bal,n_trans_hier,n_trans_tot,n_cyclic_bal,n_cyclic_hier,n_cyclic_tot,per_bal,per_hier,per_trans_bal,per_trans_hier,per_cyclic_bal,per_cyclic_hier,per_trans_bal_sep,per_cyclic_bal_sep,per_trans_hier_sep,per_cyclic_hier_sep,per_bal_sep,per_hier_sep
,String,Array…,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,030C,"[0, 0, 0, 0, 0, 0, 0, 0, 4, 21, 31, 13]",34,52,69,0,0,0,34,52,69,0.492754,0.753623,0.0,0.0,0.492754,0.753623,NaN,0.492754,NaN,0.753623,0.492754,0.753623
2,111U,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120D,"[37, 307, 15, 91, 23, 83, 71, 503, 0, 0, 0, 0]",394,470,565,394,470,1130,0,0,0,0.697345,0.831858,0.697345,0.831858,0.0,0.0,0.750442,NaN,0.913274,NaN,0.750442,0.913274
4,120C,"[7, 29, 22, 48, 21, 11, 128, 192, 2, 69, 216, 171]",217,256,458,264,425,458,240,285,458,0.473799,0.558952,0.576419,0.927948,0.524017,0.622271,0.576419,0.524017,0.927948,0.622271,0.550218,0.775109
5,201,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,111D,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,210,"[28, 111, 52, 193, 69, 193, 302, 1932, 4, 72, 278, 606]",598,165,960,598,732,2880,678,350,960,0.622917,0.171875,0.622917,0.7625,0.70625,0.364583,0.751389,0.70625,0.914931,0.364583,0.740104,0.777344
8,003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,012,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# access dataframe columns

cols = names(df_cyclic_n_arr_s[1])[startswith.(names(df_cyclic_n_arr_s[1]), "per")]

df_cyclic_n_arr_s[1][!, ["triad", cols...]]

Row,triad,per_bal,per_hier,per_trans_bal,per_trans_hier,per_cyclic_bal,per_cyclic_hier,per_trans_bal_sep,per_cyclic_bal_sep,per_trans_hier_sep,per_cyclic_hier_sep,per_bal_sep,per_hier_sep
,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,030C,0.492754,0.753623,0.0,0.0,0.492754,0.753623,NaN,0.492754,NaN,0.753623,0.492754,0.753623
2,111U,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,120D,0.697345,0.831858,0.697345,0.831858,0.0,0.0,0.750442,NaN,0.913274,NaN,0.750442,0.913274
4,120C,0.473799,0.558952,0.576419,0.927948,0.524017,0.622271,0.576419,0.524017,0.927948,0.622271,0.550218,0.775109
5,201,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,111D,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,210,0.622917,0.171875,0.622917,0.7625,0.70625,0.364583,0.751389,0.70625,0.914931,0.364583,0.740104,0.777344
8,003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
# read csv file to dataframe

# df = pd.read_csv(Path('../' + dataset + '/' + filename + '-bal_hier_all-triads-types.csv'))
# df

In [56]:
# save above dataframes

for (i, df_cyclic_n_arr) in enumerate(df_cyclic_n_arr_s)
    name = df_index.school[i] * "_class_" * string(df_index.class[i])
    CSV.write(joinpath(path, "spanish-bal-hier-all-triads-types-$name"), df_cyclic_n_arr)
    CSV.write(joinpath("spanish-bal-hier-all-triads-types-$name"), df_cyclic_n_arr)

    save(joinpath(path, "spanish-bal-hier-all-triads-types.jld2"), name, df_cyclic_n_arr)
    save(joinpath("spanish-bal-hier-all-triads-types.jld2"), name, df_cyclic_n_arr)
end

# CSV.write(joinpath(path, "spanish-bal_hier_all-triads-types.csv"), df_cyclic_n_arr)
# CSV.write(joinpath("spanish-bal_hier_all-triads-types.csv"), df_cyclic_n_arr)

# # save JLD2
# save(joinpath(path, "spanish-bal_hier_all-triads-types.jld2"), "df_cyclic_n_arr", df_cyclic_n_arr)
# save("spanish-bal_hier_all-triads-types.jld2", "df_cyclic_n_arr", df_cyclic_n_arr)

In [8]:
using JLD2
df_cyclic_n_arr = JLD2.load(joinpath(path, "soc-wiki-elec-bal_hier_all-triads-types.jld2"))["df_cyclic_n_arr"]

Row,triad,n_arr,n_bal,n_hier,n_tot,n_trans_bal,n_trans_hier,n_trans_tot,n_cyclic_bal,n_cyclic_hier,n_cyclic_tot
,String,Array…,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,030C,"[0, 0, 0, 0, 0, 0, 0, 0, 20, 426, 2309, 4040]",4466,2735,6795,0,0,0,4466,2735,6795
2,111U,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
3,120D,"[656, 2718, 804, 5957, 1529, 5232, 3536, 70680, 0, 0, 0, 0]",35682,39837,45556,35682,39837,91112,0,0,0
4,120C,"[120, 318, 252, 1857, 346, 629, 1601, 12544, 62, 812, 4918, 11875]",11919,5310,17667,13460,16786,17667,12687,5730,17667
5,201,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
6,111D,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
7,210,"[148, 598, 344, 2666, 782, 2092, 2452, 36743, 31, 413, 2943, 11888]",11150,2018,15275,11150,13025,45825,12301,3356,15275
8,003,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
9,012,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",0,0,0,0,0,0,0,0,0
